In [ ]:
import pickle
import xgboost
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from bayes_opt import BayesianOptimization

In [15]:
file = open("dataset/zone_4.pickle", "rb")
train_data = pickle.load(file)

file = open("dataset/zone_7.pickle", "rb")
test_data = pickle.load(file)

In [16]:
X_train, y_train = np.array(train_data.loc[:, train_data.columns != 'label_3m']), np.array(train_data["label_3m"]).astype(int)
train_data = None

X_test, y_test = np.array(test_data.loc[:, test_data.columns != 'label_3m']), np.array(test_data["label_3m"]).astype(int)
test_data = None

In [5]:
rus = RandomUnderSampler(.2, random_state=0)
X_train, y_train = rus.fit_resample(X_train, y_train)
print(len(X_train))

922182


In [19]:
clf =  xgboost.XGBClassifier(tree_method="gpu_hist")

In [22]:
#clf.fit(X_train, y_train)
most_important_features = np.argsort(clf.feature_importances_)[-10:]
#np.sort(clf.feature_importances_)

In [23]:
most_important_features

array([69, 43, 57,  5, 52, 30, 31, 32, 35, 34], dtype=int64)

In [7]:
pred = clf.predict(X_test)

In [8]:
cohen_kappa_score(y_test, pred)

0.5029843078079197

In [24]:
def optim_function(learning_rate=.1,
                   n_estimators=100,
                   max_depth=5,
                   min_child_weight=1,
                   gamma=0,
                   subsample=.8,
                   colsample_bytree=.8,
                   scale_pos_weight=2,
                   reg_alpha=0):
    
    max_depth = int(max_depth)
    min_child_weight = int(min_child_weight)
    n_estimators = int(n_estimators)
    
    clf = xgboost.sklearn.XGBClassifier(max_depth=int(max_depth),
                                        learning_rate=learning_rate,
                                        n_estimators=n_estimators,
                                        gamma=gamma,
                                        min_child_weight=int(min_child_weight),
                                        subsample=subsample,
                                        colsample_bytree=colsample_bytree,
                                        scale_pos_weight=scale_pos_weight,
                                        reg_alpha=reg_alpha,
                                        n_jobs=4,
                                        tree_method="gpu_hist",
                                        seed=41
                                       )
    
    clf.fit(X_train[:, most_important_features], y_train)
    
    pred = clf.predict(X_test[:, most_important_features])
    kappa = cohen_kappa_score(y_test, pred)
    return kappa

In [25]:
pbounds = {"learning_rate": (1e-4, 1e0),
           "n_estimators": (50, 500),
           "gamma": (0, .9),
           "min_child_weight": (0, 10),
           "subsample": (.1, .95),
           "colsample_bytree": (.1, .95),
           "scale_pos_weight": (1, 5),
           "max_depth": (3, 15),
           "reg_alpha": (0, 1e-3)
          }

optimizer = BayesianOptimization(
    f=optim_function,
    pbounds=pbounds,
    random_state=1,
    verbose=2,
)

In [26]:
optimizer.maximize(n_iter=100)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.4422   |  0.4545   |  0.6483   |  0.000214 |  6.628    |  1.468    |  91.55    |  0.000186 |  2.382    |  0.4373   |
|  2        |  0.481    |  0.558    |  0.3773   |  0.6853   |  5.453    |  8.781    |  62.32    |  0.000670 |  2.669    |  0.5749   |
|  3        |  0.3576   |  0.2193   |  0.1783   |  0.8008   |  14.62    |  3.134    |  361.5    |  0.000876 |  4.578    |  0.1723   |
|  4        |  0.4776   |  0.1332   |  0.1528   |  0.8782   |  4.18     |  4.211    |  481.1    |  0.000533 |  3.768    |  0.3682   |
|  5        |  0.4978   |  0.6835   |  0.7512   |  0.01839  |  12.0     |  9.889    |  386.7    |  0.000280 |  4.157    |  0.1877   |
|  6        |  0.3422   |  0.95     |  0.9      |  0.0001   | 

KeyboardInterrupt: 

In [32]:
optimizer.max

{'target': 0.5004687061446937,
 'params': {'colsample_bytree': 0.9125598867458771,
  'gamma': 0.41309970007591623,
  'learning_rate': 0.2495446719246513,
  'max_depth': 3.27021719586084,
  'min_child_weight': 0.34796142621455917,
  'n_estimators': 155.73578545537174,
  'reg_alpha': 0.0006891651003411668,
  'scale_pos_weight': 4.1957470427759915,
  'subsample': 0.42699958983347897}}

In [ ]:
def optim_function(learning_rate=.1,
                   n_estimators=100,
                   max_depth=5,
                   min_child_weight=1,
                   gamma=0,
                   subsample=.8,
                   colsample_bytree=.8,
                   scale_pos_weight=2,
                   reg_alpha=0):
    
    max_depth = int(max_depth)
    min_child_weight = int(min_child_weight)
    n_estimators = int(n_estimators)
    
    clf = xgboost.sklearn.XGBClassifier(max_depth=int(max_depth),
                                        learning_rate=learning_rate,
                                        n_estimators=n_estimators,
                                        gamma=gamma,
                                        min_child_weight=int(min_child_weight),
                                        subsample=subsample,
                                        colsample_bytree=colsample_bytree,
                                        scale_pos_weight=scale_pos_weight,
                                        reg_alpha=reg_alpha,
                                        n_jobs=4,
                                        tree_method="gpu_hist",
                                        seed=41
                                       )
    
    clf.fit(X_train[:, most_important_features], y_train)
    
    pred = clf.predict(X_test[:, most_important_features])
    kappa = cohen_kappa_score(y_test, pred)
    return kappa